In [ ]:
!pip install numpy pandas matplotlib scikit-learn nltk PyMuPDF gradio




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 584.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import fitz  # PyMuPDF
import re
import gradio as gr
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier

warnings.filterwarnings('ignore')

# Load and clean resume dataset
resumeDataSet = pd.read_csv('/content/archive (15).zip')  # Adjust the path as needed

# Clean resumes function
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]', r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

resumeDataSet['cleaned_resume'] = resumeDataSet.Resume.apply(lambda x: cleanResume(x))

# Label encoding for categories
le = LabelEncoder()
resumeDataSet['Category'] = le.fit_transform(resumeDataSet['Category'])

# Prepare features and labels
requiredText = resumeDataSet['cleaned_resume'].values
requiredTarget = resumeDataSet['Category'].values

word_vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words='english')
word_vectorizer.fit(requiredText)
WordFeatures = word_vectorizer.transform(requiredText)

X_train, X_test, y_train, y_test = train_test_split(WordFeatures, requiredTarget, random_state=42, test_size=0.2, shuffle=True, stratify=requiredTarget)

clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)

# Function to extract text and skills from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_skills(resume_text):
    skills_list = ["Python", "Java", "Machine Learning", "SQL", "JavaScript", "Data Analysis","Excel","A/B Testing","C","CPP","Leadership","Communication","Numpy","Tableau","Matplotlib","Seaborn","reactJs","Angularjs","html","css","ecplise","Bootstrap","API's","DOM","UX","UI","XML","JSON","REST",'JSP',"Node","Agile",'Scum','Version Control',"Spring","Attention to detail","Expanding Customer Base","jQuery","MongoDB","AWS","Java EE","Git","Azure","Github"]
    extracted_skills = []

    # Convert the resume text to lowercase and find skills in brackets
    resume_text = resume_text.lower()
    skills_in_brackets = re.findall(r'\((.*?)\)', resume_text)

    # Flatten the list of skills from brackets and check for skills
    for skill in skills_in_brackets:
        for sub_skill in skill.split('/'):
            sub_skill = sub_skill.strip().lower()
            if sub_skill in [s.lower() for s in skills_list] and sub_skill not in extracted_skills:
                extracted_skills.append(sub_skill)

    # Check for skills directly in the text
    for skill in skills_list:
        if skill.lower() in resume_text and skill.lower() not in extracted_skills:
            extracted_skills.append(skill.lower())

    return extracted_skills

# Main function to analyze resume
def analyze_resume(pdf_file, user_input_skills):
    resume_text = extract_text_from_pdf(pdf_file.name)
    extracted_skills = extract_skills(resume_text)

    # Predict job category
    transformed_resume = word_vectorizer.transform([resume_text])
    predicted_category = le.inverse_transform(clf.predict(transformed_resume))[0]  # Predict category from model

    # Prepare matched skills
    user_skills = [skill.strip().lower() for skill in user_input_skills.split(',')]
    matched_skills = set(extracted_skills).intersection(set(user_skills))

    return ', '.join(extracted_skills), predicted_category, ', '.join(matched_skills)  # Return all required outputs

# Gradio Interface
def main_interface(pdf_file, user_input_skills):
    extracted_skills, predicted_category, matched_skills = analyze_resume(pdf_file, user_input_skills)
    return extracted_skills, predicted_category, matched_skills  # Return all formatted outputs

# Gradio setup
inputs = [
    gr.File(label="Upload PDF Resume"),
    gr.Textbox(lines=2, placeholder="Enter required skills separated by commas...")
]
outputs = [
    gr.Textbox(label="Extracted Skills"),  # Separate output box for extracted skills
    gr.Textbox(label="Predicted Domain"),  # Separate output box for predicted category
    gr.Textbox(label="Matched Skills")  # Separate output box for matched skills
]

gr.Interface(fn=main_interface, inputs=inputs, outputs=outputs, title="HireWise").launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bcf583f595666f4597.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
